## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
def BBHE(img):
    img_hist = cv2.calcHist([img],[0], None, [256] ,[0,255])
    mean_brightness = img.mean().astype('uint8')
    H1 = img_hist[:mean_brightness]
    H2 = img_hist[mean_brightness:]
    C1 = H1.cumsum()
    C2 = H2.cumsum()
    C1_norm = C1.astype('float32')/float(np.max(C1))
    C2_norm = C2.astype('float32')/float(np.max(C2))
    C1 = (C1_norm * mean_brightness).astype('uint8')
    C2 = (C2_norm * (255 - mean_brightness)).astype('uint8') + mean_brightness
    lUT = np.array(list(C1) + list(C2))
    return cv2.LUT(img, lUT)

if not os.path.exists("jet.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/jet.bmp --no-check-certificate
jet = cv2.imread('jet.bmp', cv2.IMREAD_GRAYSCALE)
jet_BBHE = BBHE(jet)
plt.imshow(jet)
plt.title("orginal image")
plt.xticks([])
plt.yticks([])
plt.gray()
plt.show()
fig, ax = plt.subplots(1,3)
fig.set_size_inches([18,6])
ax[0].imshow(jet_BBHE)
ax[1].plot(cv2.calcHist([jet_BBHE],[0], None, [256] ,[0,255]))
ax[2].plot(cv2.calcHist([jet_BBHE],[0], None, [256] ,[0,255]).cumsum())
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title("BBHE jet.bmp")
ax[1].set_title("histogram")
ax[2].set_title("empiric dystrybuant")
plt.show()

jet_HE = cv2.equalizeHist(jet)
fig, ax = plt.subplots(1,3)
fig.set_size_inches([18,6])
ax[0].imshow(jet_HE)
ax[1].plot(cv2.calcHist([jet_HE],[0], None, [256] ,[0,255]))
ax[2].plot(cv2.calcHist([jet_HE],[0], None, [256] ,[0,255]).cumsum())
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title("HE jet.bmp")
ax[1].set_title("histogram")
ax[2].set_title("empiric dystrybuant")
plt.show()
